In [6]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# --- Page Configuration and Custom Styling ---
st.set_page_config(page_title="Stock Forecast App", layout="wide")
st.markdown("""
    <style>
        .main {background-color: #f5f5f5; color: #333;}
        h1, h2, h3 {color: #1f77b4;}
        .stButton>button {
            background-color: #1f77b4;
            color: white;
            font-weight: bold;
            border-radius: 5px;
        }
        .css-18e3th9 {background-color: #f5f5f5;}
    </style>
""", unsafe_allow_html=True)

# --- Application Title ---
st.title("📊 LSTM-Based Stock Forecast Dashboard")

# --- Sidebar Navigation ---
st.sidebar.title("Navigation Panel")
page = st.sidebar.radio("Select Section", ["Overview", "Select Company", "Data Upload", "Forecast"])

# --- Load Pretrained Model ---
model = load_model("lstm_model.h5")

# --- Utility Functions ---
def preprocess_data(data):
    df = data[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    return df, scaled, scaler

def create_dataset(data, y_col_index=3, window_size=120):
    x, y = [], []
    for i in range(window_size, len(data)):
        x.append(data[i - window_size:i])
        y.append(data[i, y_col_index])
    return np.array(x), np.array(y)

def inverse_close(pred, scaler):
    zeros = np.zeros((len(pred), 3))
    zeros_tail = np.zeros((len(pred), 1))
    pred_inversed = scaler.inverse_transform(np.concatenate([zeros, pred, zeros_tail], axis=1))[:, 3]
    return pred_inversed

def forecast_next_days(data, scaler, model, n_days=10):
    last_data = data[-120:]
    forecasts = []
    for _ in range(n_days):
        inp = last_data.reshape((1, last_data.shape[0], last_data.shape[1]))
        pred = model.predict(inp)[0]
        forecasts.append(pred)
        next_input = np.append(last_data[1:], [np.concatenate([last_data[-1][:3], pred, last_data[-1][-1:]])], axis=0)
        last_data = next_input
    return inverse_close(np.array(forecasts), scaler)

# --- Company Selector ---
selected_company = "TCS.NS"
if page == "Select Company":
    selected_company = st.selectbox("Select Company to Analyze", ["TCS.NS", "INFY.NS", "RELIANCE.NS"])
    st.session_state.selected_company = selected_company

# --- CSV Upload Section ---
data = None
if page == "Data Upload":
    uploaded = st.file_uploader("Upload your CSV file", type="csv")
    if uploaded:
        data = pd.read_csv(uploaded)
        st.success("CSV File Uploaded Successfully")

# --- Fetch Data If Not Uploaded ---
if data is None:
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    selected_company = st.session_state.get("selected_company", "TCS.NS")
    data = yf.download(selected_company, start="2010-01-01", end=today)

# --- Overview Page ---
if page == "Overview":
    st.header("📈 Recent Stock Data Preview")
    st.dataframe(data.sort_index(ascending=False).head(10))

    df, scaled_data, scaler = preprocess_data(data)
    x, y = create_dataset(scaled_data)
    x = x.reshape((x.shape[0], x.shape[1], x.shape[2]))

    y_pred_scaled = model.predict(x)
    y_pred = inverse_close(y_pred_scaled, scaler)
    y_actual = inverse_close(y.reshape(-1, 1), scaler)

    st.header("📉 Historical Forecast vs Actual")
    fig1, ax1 = plt.subplots(figsize=(14, 5))
    ax1.plot(y_actual[-700:], label="Actual Price", color='#1f77b4')
    ax1.plot(y_pred[-700:], label="Predicted Price", color='#ff7f0e')
    ax1.set_title("Actual vs Forecasted Closing Price")
    ax1.set_xlabel("Time Frame")
    ax1.set_ylabel("Price (INR)")
    ax1.grid(True)
    ax1.legend()
    st.pyplot(fig1)

    st.markdown(f"**R² Score:** {r2_score(y_actual, y_pred):.4f}  |  **RMSE:** {np.sqrt(mean_squared_error(y_actual, y_pred)):.2f}  |  **MAE:** {mean_absolute_error(y_actual, y_pred):.2f}")

    st.header("📋 Recent Forecast Summary")
    pred_df = pd.DataFrame({
        "Date": data.index[-len(y_pred):][-10:].date,
        "Actual": y_actual[-10:],
        "Predicted": y_pred[-10:]
    })
    st.table(pred_df)

# --- Forecasting Page ---
if page == "Forecast":
    st.header("🧮 Forecasting Options")
    df, scaled_data, scaler = preprocess_data(data)
    forecast_mode = st.radio("Choose Forecast Type", ["Next N Days", "Specific Date", "Custom Date Range"])

    if forecast_mode == "Next N Days":
        n = st.number_input("Forecast Horizon (Days)", min_value=1, max_value=100, value=7)
        if st.button("Generate Forecast"):
            forecasted = forecast_next_days(scaled_data, scaler, model, n)
            dates = pd.date_range(start=data.index[-1]+pd.Timedelta(days=1), periods=n, freq='B')
            result = pd.DataFrame({"Date": dates.date, "Predicted Close": forecasted})
            st.dataframe(result)
            csv = result.to_csv(index=False).encode('utf-8')
            st.download_button("Download Results", csv, "forecast.csv", "text/csv")

    elif forecast_mode == "Specific Date":
        date_input = st.date_input("Select a Future Date")
        st.info("Prediction is based on extrapolation from the last available data point.")
        if st.button("Predict Date"):
            days_gap = (date_input - data.index[-1].date()).days
            if days_gap > 0:
                forecasted = forecast_next_days(scaled_data, scaler, model, days_gap)
                st.success(f"Forecasted Close on {date_input.strftime('%Y-%m-%d')}: ₹{forecasted[-1]:.2f}")
            else:
                st.error("Date must be later than the last date in the dataset.")

    elif forecast_mode == "Custom Date Range":
        start_date = st.date_input("From Date")
        end_date = st.date_input("To Date")
        if start_date and end_date and end_date > start_date:
            if st.button("Predict Range"):
                days_gap = (start_date - data.index[-1].date()).days
                range_days = (end_date - start_date).days + 1
                if days_gap >= 0:
                    forecasted = forecast_next_days(scaled_data, scaler, model, days_gap + range_days)
                    forecasted = forecasted[days_gap:]
                    dates = pd.date_range(start=start_date, periods=range_days, freq='B')
                    result = pd.DataFrame({"Date": dates.date, "Predicted Close": forecasted})
                    st.dataframe(result)
                    csv = result.to_csv(index=False).encode('utf-8')
                    st.download_button("Export CSV", csv, "forecast.csv", "text/csv")
                else:
                    st.error("The selected start date must be beyond the dataset's range.")
        else:
            st.warning("Please ensure valid start and end dates are selected.")


2025-06-25 16:17:39.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:39.148 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:39.149 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:39.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:39.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:39.152 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:39.152 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:39.154 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

116/116 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step


2025-06-25 16:17:59.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:59.906 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:59.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:17:59.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:18:00.436 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:18:00.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:18:00.440 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-25 16:18:00.445 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar